### Import Libraries

In [ ]:
# import required libraries
import kfp
from typing import Any
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, ClassificationMetrics,
                        Metrics, component)
import sys
import os
import re

from pathlib import Path
from datetime import date
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import google
from google.oauth2 import credentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from google.cloud import storage
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components.v1.batch_predict_job import \
    ModelBatchPredictOp as batch_prediction_op

# Set global vars
pth_project = Path(os.getcwd())
sys.path.insert(0, pth_project.as_posix())

### YAML Parameters

In [ ]:
#tag cell with parameters
PROJECT_ID=''
DATASET_ID=''
OUTPUT_DATASET_ID=''
TABLE_ID=''
SCORE_TABLE_ID=''
RESOURCE_BUCKET=''
FILE_BUCKET=''
REGION=''
MODEL_ID=''
STACK_NAME=''
MODEL_NAME=''
SERVICE_TYPE=''
SERVICE_TYPE_NAME=''
PIPELINE_TYPE=''
TRAINING_PIPELINE_PATH=''
SERVING_PIPELINE_PATH=''
HS_NBA_UTILS_PATH=''
MODEL_TYPE=''
LOAD_SQL=''
PREPROCESS_OUTPUT_CSV=''
SAVE_FILE_NAME=''
STATS_FILE_NAME=''

AGGREGATE_RESULTS = True
AGGREGATE_RESULTS_TABLE_ID = 'bq_product_recommendation_ranked'

In [ ]:
#tag cell with parameters
PROJECT_ID =  'divg-groovyhoon-pr-d2eab4'
DATASET_ID = 'nba_features_prospect'
OUTPUT_DATASET_ID = 'telus_ffh_nba'
TABLE_ID = 'master_features_set_prospect_predict_vw'
SCORE_TABLE_ID = 'bq_hs_nba_prospects_scores'
RESOURCE_BUCKET = 'divg-groovyhoon-pr-d2eab4-default'
FILE_BUCKET = 'divg-groovyhoon-pr-d2eab4'
REGION = 'northamerica-northeast1'
MODEL_ID = '7002'
STACK_NAME = 'ffh_nba'
MODEL_NAME = 'hs_nba_prospects'
SERVICE_TYPE = 'hs_nba_prospects'
SERVICE_TYPE_NAME = 'hs-nba-prospects'
PIPELINE_TYPE='serving_pipeline'
TRAINING_PIPELINE_PATH = 'vertex_pipelines/hs_nba_prospects/training_pipeline'
SERVING_PIPELINE_PATH = 'vertex_pipelines/hs_nba_prospects/serving_pipeline'
HS_NBA_UTILS_PATH =  'vertex_pipelines/hs_nba_utils/notebook'
MODEL_TYPE='acquisition'
LOAD_SQL='load_serving_data.sql'
PREPROCESS_OUTPUT_CSV='df_serving.csv' 
SAVE_FILE_NAME='df_test_exp.csv'
STATS_FILE_NAME='df_stats.csv'

AGGREGATE_RESULTS = True
AGGREGATE_RESULTS_TABLE_ID = 'bq_product_recommendation_ranked'

### Pipeline Parameters

In [ ]:
TRAINING_PIPELINE_NAME = f'{SERVICE_TYPE_NAME}-train-pipeline' # Same name as pulumi.yaml
SERVING_PIPELINE_NAME = f'{SERVICE_TYPE_NAME}-serving-pipeline' # Same name as pulumi.yaml
TRAINING_PIPELINE_DESCRIPTION = f'{SERVICE_TYPE_NAME}-train-pipeline'
SERVING_PIPELINE_DESCRIPTION = f'{SERVICE_TYPE_NAME}-serving-pipeline'
PIPELINE_ROOT = f"gs://{FILE_BUCKET}"
REGION = "northamerica-northeast1"

### Import kfp components

In [ ]:
def extract_dir_from_bucket(
    bucket: Any, local_path: Path, prefix: str, split_prefix: str = 'serving_pipeline' 
):        
    """
    Download files from a specified bucket to a local path, excluding a specified prefix.

    Parameters:
      - bucket: The bucket object from which to download files.
      - local_path: The local path where the files will be downloaded to.
      - prefix: The prefix to filter the files in the bucket. Only files with this prefix will be downloaded.
      - split_prefix: The prefix to exclude from the downloaded file paths. Default is 'serving_pipeline'.
    """
    for blob in bucket.list_blobs(prefix=prefix):
        if not blob.name.endswith("/"):
            path = local_path / blob.name.split(f'{split_prefix}/')[-1]
            str_path = path.as_posix()
            Path(str_path[:str_path.rindex('/')]).mkdir(parents=True, exist_ok=True)
            blob.download_to_filename(str_path)

In [ ]:
# download pipeline kfp components locally
storage_client = storage.Client(PROJECT_ID)
bucket = storage_client.bucket(RESOURCE_BUCKET)
prefix_core = f'{STACK_NAME}/{SERVING_PIPELINE_PATH}/kfp_components'
extract_dir_from_bucket(bucket, pth_project, prefix_core)

# import kfp components
from kfp_components.preprocess import preprocess
from kfp_components.predict import predict
from kfp_components.postprocess import postprocess
from kfp_components.aggregate_results import aggregate_results

### Date Parameters

In [ ]:
# training pipeline is to run on the 3rd of every month
# change the training time window every 3rd of month to R12 months as of {last day of 2 months ago} e.g. on May 1st, training window to be 2023-04-01 to 2024-03-31

# set training dates
trainingDate = (date.today() - relativedelta(days=2))

# training dates
TRAIN_DATE = trainingDate.strftime('%Y%m%d')  # date.today().strftime('%Y%m%d')
TRAIN_DATE_DASH = trainingDate.strftime('%Y-%m-%d')

FROM_DATE = trainingDate.replace(day=1) + relativedelta(months=-13)
TO_DATE = trainingDate.replace(day=1) + relativedelta(months=-1, days=-1)

FROM_DATE = FROM_DATE.strftime('%Y-%m-%d')
TO_DATE = TO_DATE.strftime('%Y-%m-%d')

### Pipeline

In [ ]:
# library imports
from kfp.v2 import compiler
from google.cloud.aiplatform import pipeline_jobs
@dsl.pipeline(
    name=SERVING_PIPELINE_NAME, 
    description=SERVING_PIPELINE_DESCRIPTION
    )
def pipeline(
        project_id: str = PROJECT_ID,
        region: str = REGION,
        resource_bucket: str = RESOURCE_BUCKET, 
        file_bucket: str = FILE_BUCKET
    ):
    
    #### this code block is only for a personal workbench 
    
    import google.oauth2.credentials
    token = !gcloud auth print-access-token
    token_str = token[0]
    
    #### the end
    
    from datetime import datetime
    update_ts = datetime.now()
    update_ts_str = update_ts.strftime('%Y-%m-%d %H:%M:%S')
    
    from pathlib import Path
    cwd = os.getcwd() 
    
#     # ----- preprocessing train data --------
#     preprocess_op = preprocess(project_id=PROJECT_ID,
#                         dataset_id=DATASET_ID,
#                         table_id=TABLE_ID,
#                         file_bucket=FILE_BUCKET, 
#                         resource_bucket=RESOURCE_BUCKET, 
#                         stack_name=STACK_NAME,
#                         pipeline_path=SERVING_PIPELINE_PATH,
#                         hs_nba_utils_path=HS_NBA_UTILS_PATH,
#                         model_type=MODEL_TYPE,
#                         pipeline_type=PIPELINE_TYPE, 
#                         load_sql=LOAD_SQL,
#                         preprocess_output_csv=PREPROCESS_OUTPUT_CSV,
#                         token=token_str
#                                     )

#     preprocess_op.set_memory_limit('64G')
#     preprocess_op.set_cpu_limit('8')

#     predict_op = predict(project_id=PROJECT_ID,
#                         resource_bucket=RESOURCE_BUCKET,
#                         file_bucket=FILE_BUCKET,
#                         stack_name=STACK_NAME,
#                         service_type=SERVICE_TYPE, 
#                         model_type=MODEL_TYPE,
#                         pipeline_type=PIPELINE_TYPE, 
#                         training_pipeline_path=TRAINING_PIPELINE_PATH,
#                         serving_pipeline_path=SERVING_PIPELINE_PATH,
#                         preprocess_output_csv=PREPROCESS_OUTPUT_CSV ,
#                         hs_nba_utils_path=HS_NBA_UTILS_PATH
#                         )
    
#     predict_op.set_memory_limit('64G')
#     predict_op.set_cpu_limit('8')

    postprocess_op = postprocess(project_id=PROJECT_ID,
                        output_dataset_id=OUTPUT_DATASET_ID,
                        score_table_id=SCORE_TABLE_ID,
                        resource_bucket=RESOURCE_BUCKET,
                        file_bucket=FILE_BUCKET,
                        stack_name=STACK_NAME,
                        model_type=MODEL_TYPE,
                        pipeline_type=PIPELINE_TYPE, 
                        pipeline_path=SERVING_PIPELINE_PATH,
                        hs_nba_utils_path=HS_NBA_UTILS_PATH, 
                        token=token_str
                        )
    
    postprocess_op.set_memory_limit('64G')
    postprocess_op.set_cpu_limit('4')

#     if AGGREGATE_RESULTS:
#         aggregate_results_op = aggregate_results(
#             project_id=PROJECT_ID,
#             dataset_id=DATASET_ID,
#             aggregate_results_table_id=AGGREGATE_RESULTS_TABLE_ID,
#             resource_bucket=RESOURCE_BUCKET,
#             stack_name=STACK_NAME,
#             pipeline_path=SERVING_PIPELINE_PATH,
#             hs_nba_utils_path=HS_NBA_UTILS_PATH
#         )
#         aggregate_results_op.set_memory_limit('64G')
#         aggregate_results_op.set_cpu_limit('4')


    predict_op.after(preprocess_op)
    postprocess_op.after(predict_op)
    # if AGGREGATE_RESULTS:
    #     aggregate_results_op.after(postprocess_op)
    
    # postprocess_op

### Run the Pipeline Job

In [ ]:
# from kfp.v2 import compiler
# from google.cloud.aiplatform import pipeline_jobs

# compiler.Compiler().compile(
#    pipeline_func=hs_nba_existing_customers_pipeline, package_path="pipeline.json"
# )

# job = pipeline_jobs.PipelineJob(
#    display_name='hs-nba-existing-customers-pipeline',
#    template_path="pipeline.json",
#    location=REGION,
#    enable_caching=False,
#    pipeline_root = f"gs://{RESOURCE_BUCKET}"
# )
# job.run(service_account = f"bilayer-sa@{PROJECT_ID}.iam.gserviceaccount.com")


In [ ]:
import google.oauth2.credentials
import json

token = !gcloud auth print-access-token
CREDENTIALS = google.oauth2.credentials.Credentials(token[0])

compiler.Compiler().compile(
   pipeline_func=pipeline, package_path="pipeline.json"
)

job = pipeline_jobs.PipelineJob(
   display_name=TRAINING_PIPELINE_NAME,
   template_path="pipeline.json",
   credentials = CREDENTIALS,
   pipeline_root = PIPELINE_ROOT,
   location=REGION,
   enable_caching=False # I encourage you to enable caching when testing as it will reduce resource use
)

job.run()

In [ ]:
test_dict = {} 

test_dict[0] = 'value1'

print(test_dict)